<a href="https://colab.research.google.com/github/nemodesouza/TCC-AIX/blob/main/PA3_validade_convergente_discriminante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# UNIVERSIDADE FEDERAL DE SANTA CATARINA
# CENTRO TECNOLÓGICO
# DEPARTAMENTO DE INFORMÁTICA E ESTATÍSTICA
# GRADUAÇÃO EM SISTEMAS DE INFORMAÇÃO
# TRABALHO DE CONCLUSÃO DE CURSO
# NEMO DE SOUZA

# PA3 = Análise de Validade Convergente e Discriminante do Checklist AIX
# Script para a análise de validade do checklist de heurísticas de AIX

# Carregamento de bibliotecas
if (!require(psych)) install.packages("psych")        # Para análise fatorial
if (!require(polycor)) install.packages("polycor")    # Para correlação policórica
if (!require(corrplot)) install.packages("corrplot")  # Para visualização de correlações

library(psych)
library(polycor)
library(corrplot)

In [ ]:
# Carregamento dos dados
dados <- "PA2 a PA4 - matrix_avaliacao_aplicativos.csv"
heuristicas <- "matriz_heuristicas_e_itens_checklist.csv"

dados <- read.csv(dados, stringsAsFactors = FALSE)
heuristicas <- read.csv(heuristicas, stringsAsFactors = FALSE)

# Verificação da estrutura dos dados
str(dados)
head(dados)

# Verificar a estrutura das heurísticas
str(heuristicas)
head(heuristicas)

In [ ]:
#### Preparação dos dados para análise

# Seleção das colunas dos itens (c1 a c23)
itens <- dados[, grep("^c\\d+$", names(dados))]

# Conversão das respostas para valores numéricos (Sim = 1, Não = 0, NA = NA)
itens_num <- as.data.frame(lapply(itens, function(x) {
  ifelse(x == "Sim", 1, ifelse(x == "Não", 0, NA))
}))

# Verificação da conversão
head(itens_num)

In [ ]:
#### Criação do dicionário para mapear códigos de itens para descrições de heurísticas

# Estrutura arquivo:
# - ID Heurística: h1, h2, etc.
# - Heurística: Descrição da heurística
# - ID checklist: c1, c2, etc.
# - Item de checklist: Descrição do checklist

id_heuristicas <- heuristicas$ID.Heurística
descricoes_heuristicas <- heuristicas$Heurística
codigos_itens <- heuristicas$ID.checklist
descricoes_itens <- heuristicas$Item.de.checklist

# Criação dataframe de mapeamento
dicionario <- data.frame(
  codigo_item = codigos_itens,
  descricao_item = descricoes_itens,
  id_heuristica = id_heuristicas,
  descricao_heuristica = descricoes_heuristicas,
  stringsAsFactors = FALSE
)

# Verificação o dicionário criado
head(dicionario)

In [ ]:
#### Identificação e remoção de colunas problemáticas

# Cálculo da porcentagem de NAs em cada coluna
na_percentagem <- colMeans(is.na(itens_num)) * 100
print("Porcentagem de NAs em cada coluna:")
print(na_percentagem)

# Cálculo do desvio padrão de cada coluna
desvios <- sapply(itens_num, sd, na.rm = TRUE)
print("Desvio padrão de cada coluna:")
print(desvios)

# Colunas com desvio padrão zero ou muito baixo
colunas_desvio_zero <- names(desvios[desvios < 0.1])
print("Colunas com desvio padrão próximo de zero:")
print(colunas_desvio_zero)

# Colunas com muitos NAs (mais de 30%)
colunas_muitos_na <- names(na_percentagem[na_percentagem > 30])
print("Colunas com mais de 30% de NAs:")
print(colunas_muitos_na)

# Colunas a serem removidas (desvio zero ou muitos NAs)
colunas_remover <- unique(c(colunas_desvio_zero, colunas_muitos_na))
print("Colunas a serem removidas:")
print(colunas_remover)

# Conjunto de dados limpo
itens_limpos <- itens_num[, !(names(itens_num) %in% colunas_remover)]

# Verificação das dimensões do conjunto de dados após limpeza
print("Dimensões do conjunto de dados após limpeza:")
print(dim(itens_limpos))

# Dataframe com informações detalhadas sobre os itens removidos
itens_removidos <- data.frame(
  codigo = colunas_remover,
  proporcao_na = na_percentagem[colunas_remover],
  id_heuristica = sapply(colunas_remover, function(codigo) {
    idx <- which(dicionario$codigo_item == codigo)
    if (length(idx) > 0) dicionario$id_heuristica[idx[1]] else NA
  }),
  descricao_heuristica = sapply(colunas_remover, function(codigo) {
    idx <- which(dicionario$codigo_item == codigo)
    if (length(idx) > 0) dicionario$descricao_heuristica[idx[1]] else NA
  }),
  motivo = sapply(colunas_remover, function(codigo) {
    if (codigo %in% colunas_desvio_zero) {
      if (codigo %in% colunas_muitos_na) {
        return("Desvio padrão próximo de zero e muitos NAs")
      } else {
        return("Desvio padrão próximo de zero")
      }
    } else {
      return("Muitos valores NA")
    }
  })
)

# Ordenação por proporção de NA
itens_removidos <- itens_removidos[order(-itens_removidos$proporcao_na), ]

# Informações detalhadas sobre os itens removidos
print("Detalhes dos itens removidos:")
print(itens_removidos)


In [ ]:
#### Análise de Validade Convergente e Discriminante

# Verificação da distribuição das respostas
summary(itens_limpos)
apply(itens_limpos, 2, table)

# Verificar tamanho da amostra
nrow(itens_limpos)
ncol(itens_limpos)

# Regra geral: pelo menos 5-10 casos por variável
cat("Razão casos/variáveis:", nrow(itens_limpos)/ncol(itens_limpos))

# Identificar variáveis com pouca variação
variancia <- apply(itens_limpos, 2, var, na.rm = TRUE)
variaveis_problematicas <- names(variancia[variancia < 0.1])
print(variaveis_problematicas)

# Remover se necessário
itens_filtrados <- itens_limpos[, !names(itens_limpos) %in% variaveis_problematicas]


# Cálculo da matriz de correlação policórica
matriz_policorica <- polychoric(itens_limpos,
                                smooth = TRUE,
                                global = FALSE)


In [ ]:
# Salvamento da matriz de correlação policórica
write.csv(matriz_policorica$rho, "matriz_correlacao_policorica.csv")

# Visualização da matriz de correlação policórica
print("Matriz de correlação policórica:")
print(round(matriz_policorica$rho, 2))

In [ ]:
# Matriz de correlação policórica
corrplot(matriz_policorica_smooth$rho, method = "color", type = "upper",
         tl.col = "black", tl.srt = 45,
         title = "Matriz de Correlação Policórica",
         mar = c(0, 0, 1, 0))

In [ ]:
#### Análise de validade convergente e discriminante

# Identificação de itens que pertencem à mesma heurística

# Criação de uma lista de heurísticas válidas (que têm itens no conjunto limpo)
heuristicas_validas <- list()
for (h in unique(dicionario$id_heuristica)) {
  itens_h <- dicionario$codigo_item[dicionario$id_heuristica == h]
  itens_validos_h <- intersect(itens_h, names(itens_limpos))
  if (length(itens_validos_h) > 0) {
    heuristicas_validas[[h]] <- itens_validos_h
  }
}

# Exibição das heurísticas válidas após filtragem
print("Heurísticas válidas após filtragem:")
print(heuristicas_validas)

In [ ]:
# Cálculo da média das correlações dentro de cada heurística (validade convergente)
validade_convergente <- list()
for (nome in names(heuristicas_validas)) {
  itens <- heuristicas_validas[[nome]]
  if (length(itens) > 1) {  # Precisa de pelo menos 2 itens para calcular correlação
    correlacoes <- matriz_policorica$rho[itens, itens]
    # Remoção da diagonal (correlações de um item com ele mesmo)
    diag(correlacoes) <- NA
    media_correlacao <- mean(correlacoes, na.rm = TRUE)
    validade_convergente[[nome]] <- media_correlacao
  } else {
    validade_convergente[[nome]] <- NA  # Não é possível calcular com apenas 1 item
  }
}

# Cálculo da média das correlações entre itens de diferentes heurísticas (validade discriminante)
validade_discriminante <- matrix(NA, nrow = length(heuristicas_validas),
                               ncol = length(heuristicas_validas))
rownames(validade_discriminante) <- names(heuristicas_validas)
colnames(validade_discriminante) <- names(heuristicas_validas)

for (i in 1:length(heuristicas_validas)) {
  nome_i <- names(heuristicas_validas)[i]
  itens_i <- heuristicas_validas[[nome_i]]

  for (j in 1:length(heuristicas_validas)) {
    nome_j <- names(heuristicas_validas)[j]
    itens_j <- heuristicas_validas[[nome_j]]

    if (i != j && length(itens_i) > 0 && length(itens_j) > 0) {
      correlacoes <- matriz_policorica$rho[itens_i, itens_j]
      media_correlacao <- mean(correlacoes, na.rm = TRUE)
      validade_discriminante[i, j] <- media_correlacao
    }
  }
}

# Conversão para dataframe para facilitar a visualização
validade_convergente_df <- data.frame(
  Heuristica = names(validade_convergente),
  Media_Correlacao_Interna = unlist(validade_convergente),
  stringsAsFactors = FALSE
)

# Adição da descrição da heurística
validade_convergente_df$Descricao_Heuristica <- sapply(validade_convergente_df$Heuristica, function(h) {
  idx <- which(dicionario$id_heuristica == h)
  if (length(idx) > 0) dicionario$descricao_heuristica[idx[1]] else NA
})



In [ ]:
# Salvamento dos resultados
write.csv(validade_convergente_df, "validade_convergente.csv", row.names = FALSE)
write.csv(validade_discriminante, "validade_discriminante.csv")


In [ ]:
# Visualização dos resultados de validade convergente
print("Validade Convergente (média das correlações dentro de cada heurística):")
print(validade_convergente_df)

# Visualização dos resultados de validade discriminante
print("Validade Discriminante (média das correlações entre heurísticas diferentes):")
print(round(validade_discriminante, 2))

In [ ]:
# Cálculo da média das correlações com outras heurísticas para cada heurística
media_correlacoes_externas <- rowMeans(validade_discriminante, na.rm = TRUE)
validade_comparacao <- data.frame(
  Heuristica = names(heuristicas_validas),
  Descricao = sapply(names(heuristicas_validas), function(h) {
    idx <- which(dicionario$id_heuristica == h)
    if (length(idx) > 0) dicionario$descricao_heuristica[idx[1]] else NA
  }),
  Convergente = unlist(validade_convergente),
  Discriminante = media_correlacoes_externas,
  stringsAsFactors = FALSE
)

# Salvamento da comparação
write.csv(validade_comparacao, "validade_comparacao.csv", row.names = FALSE)

# Visualização da comparação
print("Comparação entre Validade Convergente e Discriminante:")
print(validade_comparacao)

In [ ]:
#### Análise Fatorial Exploratória (AFE) para verificar a estrutura latente dos itens

# Verificação da adequação dos dados para AFE
kmo_result <- KMO(matriz_policorica$rho)
print("Índice KMO (Kaiser-Meyer-Olkin):")
print(kmo_result$MSA)

In [ ]:
# Teste de Bartlett para esfericidade
bartlett_test <- cortest.bartlett(matriz_policorica$rho, n = nrow(itens_limpos))
print("Teste de Bartlett para esfericidade:")
print(bartlett_test)

In [ ]:
# Determinação do número ideal de fatores
scree_data <- fa.parallel(matriz_policorica$rho, n.obs = nrow(itens_limpos), fa = "fa")


In [ ]:
# Número sugerido de fatores
n_fatores <- scree_data$nfact
print("Número sugerido de fatores:", n_fatores)

# Realização da análise fatorial exploratória
afe_resultado <- fa(matriz_policorica$rho, nfactors = n_fatores, rotate = "varimax", fm = "ml")
print(afe_resultado)

In [ ]:
# Salvamento dos resultados da AFE
sink("afe_resultados.txt")
print(afe_resultado)
sink()

In [ ]:
# Cálculo da variância explicada por cada fator
var_explicada <- afe_resultado$Vaccounted
write.csv(var_explicada, "variancia_explicada.csv")

# Exibição da variância explicada
print("Variância explicada por fator:")
print(var_explicada)



In [ ]:
# Interpretação dos fatores
# Identificação dos itens com cargas significativas em cada fator
cargas_significativas <- afe_resultado$loadings > 0.3
itens_por_fator <- list()

for (i in 1:n_fatores) {
  itens_fator <- rownames(afe_resultado$loadings)[cargas_significativas[, i]]
  itens_por_fator[[paste0("Fator", i)]] <- itens_fator

  print(paste0("Itens com cargas significativas no Fator ", i, ":"))
  for (item in itens_fator) {
    idx <- which(dicionario$codigo_item == item)
    if (length(idx) > 0) {
      heuristica <- dicionario$id_heuristica[idx]
      descricao_h <- dicionario$descricao_heuristica[idx]
      print(paste0(item, " (", heuristica, " - ", descricao_h, ")"))
    } else {
      print(item)
    }
  }
  print("")
}

# Salvamento da estrutura de fatores
itens_por_fator_df <- data.frame(
  Fator = rep(names(itens_por_fator), sapply(itens_por_fator, length)),
  Item = unlist(itens_por_fator),
  stringsAsFactors = FALSE
)

itens_por_fator_df$Heuristica <- sapply(itens_por_fator_df$Item, function(item) {
  idx <- which(dicionario$codigo_item == item)
  if (length(idx) > 0) dicionario$id_heuristica[idx[1]] else NA
})

itens_por_fator_df$Descricao_Heuristica <- sapply(itens_por_fator_df$Item, function(item) {
  idx <- which(dicionario$codigo_item == item)
  if (length(idx) > 0) dicionario$descricao_heuristica[idx[1]] else NA
})

write.csv(itens_por_fator_df, "estrutura_fatores.csv", row.names = FALSE)
